# Customer Purchase Behaviour & Sentiment Analysis

## Project Objective
The objective of this project is to analyse customer purchase behaviour and review sentiment to understand customer satisfaction levels, product perception, and engagement trends. This analysis will help identify patterns in how customers interact with products and how their feedback reflects their overall experience.

---

## What This Project Helps Us Discover
- Which products customers are most satisfied or dissatisfied with  
- Overall customer sentiment trends over time  
- Customer behaviour patterns reflected through reviews  
- Insights that can support business decision-making and service improvement  

---

## Project Flow
1. Data cleaning in Python to prepare and process the raw review data  
2. SQL analysis in PostgreSQL to extract business insights from the dataset  
3. Data visualisation in Python to present key findation interpretation  
r interpretation   interpretation  


In [2]:
import sys

# install required packages in the SAME environment Jupyter is using
!{sys.executable} -m pip install textblob nltk pandas matplotlib seaborn


   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
    --------------------------------------- 10.2/624.3 kB ? eta -:--:--
   --- ------------------------------------ 61.4/624.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  614.4/624.3 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 624.3/624.3 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------------------------------------- - 1.5/1.5 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishaa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ishaa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
from textblob import TextBlob
TextBlob("The product is amazing!").sentiment  # to check if code works

Sentiment(polarity=0.7500000000000001, subjectivity=0.9)

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob   # for sentiment analysis
import re   # for cleaning text (regular expressions module)
import nltk  # Natural Language Toolkit to provide tool 'stopwords'
nltk.download('stopwords')  # Downloads English stopwords 
from nltk.corpus import stopwords   # lets us remove common filter words

sns.set()  # makes visualization looks better

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df = pd.read_csv("customer_reviews_data.csv")

In [10]:
df.head()

,ReviewID,CustomerID,ProductID,ReviewText,ReviewDate
0,1,486,267,So impressed by the quality. This product trul...,5/12/2024
1,2,810,246,I'm very happy with the performance. It does e...,3/7/2024
2,3,855,291,I regret buying this. The quality is terrible ...,11/15/2023
3,4,524,235,"It serves its purpose, but it's not anything e...",5/27/2024
4,5,238,220,Very high-quality product. I would buy it agai...,9/10/2023


In [11]:
df.shape

(1000, 5)

In [22]:
df.describe(include='all')

,ReviewID,CustomerID,ProductID,ReviewText,ReviewDate
count,1000.000000,1000.000000,1000.000000,1000,1000
unique,NaN,NaN,NaN,189,NaN
top,NaN,NaN,NaN,Very high-quality product. I would buy it agai...,NaN
freq,NaN,NaN,NaN,15,NaN
mean,500.500000,554.596000,248.977000,NaN,2023-12-24 17:11:02.400000
min,1.000000,100.000000,200.000000,NaN,2023-06-26 00:00:00
25%,250.750000,343.500000,223.000000,NaN,2023-09-29 00:00:00
50%,500.500000,563.500000,249.000000,NaN,2023-12-25 00:00:00
75%,750.250000,744.750000,275.000000,NaN,2024-03-27 00:00:00
max,1000.000000,999.000000,299.000000,NaN,2024-06-25 00:00:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ReviewID    1000 non-null   int64 
 1   CustomerID  1000 non-null   int64 
 2   ProductID   1000 non-null   int64 
 3   ReviewText  1000 non-null   object
 4   ReviewDate  1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [13]:
df.isnull().sum()

ReviewID      0
CustomerID    0
ProductID     0
ReviewText    0
ReviewDate    0
dtype: int64

In [15]:
(df['ReviewText'].str.strip() == "").sum() # to check null string valus

0

In [16]:
# to check if ReviewDate column is datetime type
# the datetime type columns hepls in sorting by date, extract year/month/date
# or to analyze trends over time
df.dtypes

ReviewID       int64
CustomerID     int64
ProductID      int64
ReviewText    object
ReviewDate    object
dtype: object

In [18]:
df['ReviewDate'] = pd.to_datetime(df['ReviewDate'])

In [19]:
df.dtypes

ReviewID               int64
CustomerID             int64
ProductID              int64
ReviewText            object
ReviewDate    datetime64[ns]
dtype: object

In [21]:
# to check if duplicate reiews exists
df['ReviewID'].duplicated().sum()

0

In [23]:
# install contraction to covert "don't" to "do not"
import sys
!{sys.executable} -m pip install contractions


   ---------------------------------------- 0.0/345.1 kB ? eta -:--:--
   ------------------- ------------------- 174.1/345.1 kB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 345.1/345.1 kB 5.4 MB/s eta 0:00:00


In [25]:
# Text Cleaning

import re
import contractions
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

negation_words = {"not", "no", "don't", "doesn't", "didn't", "isn't",
                  "wasn't", "weren't", "won't", "can't", "couldn't", 
                  "shouldn't", "haven't", "hasn't", "hadn't"}

for w in negation_words:
    stop_words.discard(w)

In [28]:
def clean_text(text):

    # ensure string in lowercase
    text = str(text).lower()
    # expand contractions
    text = contractions.fix(text)
    # remove exerything except letters + spaces
    text = re.sub(r'[^a-z\s]', ' ', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # remove stopwords EXCEPT negations
    text = " ".join(
    word for word in text.split()
    if word not in stop_words
    ) 
    return text

# Apply to the dataset
df['CleanReview'] = df['ReviewText'].apply(clean_text)

In [29]:
df.dtypes

ReviewID                int64
CustomerID              int64
ProductID               int64
ReviewText             object
ReviewDate     datetime64[ns]
CleanReview            object
dtype: object

In [30]:
# creating columns for various functions
# to count words in cleaned review
df['ReviewLength'] = df['CleanReview'].apply(lambda x: len(x.split()))
df['Year'] = df['ReviewDate'].dt.year  # To extract year
df['Month'] = df['ReviewDate'].dt.month  # to extract month
df['Weekday'] = df ['ReviewDate'].dt.day_name() # to extract weekday

df.dtypes

ReviewID                 int64
CustomerID               int64
ProductID                int64
ReviewText              object
ReviewDate      datetime64[ns]
CleanReview             object
ReviewLength             int64
Year                     int32
Month                    int32
Weekday                 object
dtype: object

In [31]:
# Create sentiment score
from textblob import TextBlob   # library for sentiment

def sentiment_score(text):
    return TextBlob(text).sentiment.polarity  # provide sentiment score

df['SentimentScore'] = df['CleanReview'].apply(sentiment_score)

In [33]:
# Convert sentiment score into sentiment label
def label_sentiment(score):
    if score > 0.1:
        return "Positive"
    elif score < -0.1:
        return "Negative"
    else:
        return "Neutral"

df['SentimentLabel'] = df['SentimentScore'].apply(label_sentiment)


In [34]:
df.head()

,ReviewID,CustomerID,ProductID,ReviewText,ReviewDate,CleanReview,ReviewLength,Year,Month,Weekday,SentimentScore,SentimentLabel
0,1,486,267,So impressed by the quality. This product trul...,2024-05-12,impressed quality product truly delivers,5,2024,5,Sunday,1.000000,Positive
1,2,810,246,I'm very happy with the performance. It does e...,2024-03-07,happy performance exactly need,4,2024,3,Thursday,0.525000,Positive
2,3,855,291,I regret buying this. The quality is terrible ...,2023-11-15,regret buying quality terrible stopped working...,8,2023,11,Wednesday,-1.000000,Negative
3,4,524,235,"It serves its purpose, but it's not anything e...",2024-05-27,serves purpose not anything extraordinary,5,2024,5,Monday,0.333333,Positive
4,5,238,220,Very high-quality product. I would buy it agai...,2023-09-10,high quality product would buy without hesitation,7,2023,9,Sunday,0.160000,Positive


In [35]:
# export cleaned dataset to CSV file
df.to_csv("cleaned_customer_reviews_sentiment.csv", index = False)